In [53]:
# Le module selenium.webdriver fournit toutes les implémentations de WebDriver. Les implémentations WebDriver 
# actuellement prises en charge sont Firefox, Chrome, Safari, Edge. La classe Keys fournit des touches dans 
# le clavier telles que RETURN, F1, ALT etc.
# la classe By est pour localiser les éléments
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Option va nous permettre d'inclure des paramètres supplémentaires,
# l'ouverture d'une page web pleine écran par exemple
from selenium.webdriver.chrome.options import Options

# une autre librairie python utile "sleep" va nous permettre d'effectuer des temps d'arrêt,
# pour imiter le comportement humain.
from time import sleep
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd

In [54]:
# on créé une option plein écran
options = Options()
options.add_argument("--no-sandbox")

# on charge le driver en incluant les options
driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')

# on ouvre la page
driver.get('https://www.forexfactory.com/trades')

# on marque un temps d'arrêt
sleep(3)

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_14904\1595881486.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')
C:\Users\utilisateur\AppData\Local\Temp\ipykernel_14904\1595881486.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')


In [55]:
cells = driver.find_elements(By.XPATH,'//td[@class="trade trades_activity__cell trades_activity__cell--trade"]')
sleep(1)

In [56]:
trade_list = []
for p in range(len(cells)):
    trade_list.append(cells[p].text)

In [57]:
names = driver.find_elements(By.XPATH,'//td[@class="trader trades_activity__cell trades_activity__cell--trader"]')

In [58]:
names_list = []
for p in range(len(names)):
    names_list.append(names[p].text)

In [59]:
data_tuples = list(zip(trade_list[1:],names_list[1:])) 
columns=['Trade','Name']

In [60]:
new_df = pd.DataFrame(data_tuples, columns=columns) 

In [61]:
new_df['Pair'] = new_df['Trade'].apply(lambda x: x.split(" ", 1)[0])

In [62]:
new_df['Position'] = new_df['Trade'].apply(lambda x: 'BUY' or 'SELL' if 'BUY'  in x else 'SELL')

In [63]:
new_df['Trade'] = new_df['Trade'].str.slice(start=10) 
new_df['price_level'] =  new_df['Trade'].str.extract(r"(\d+\.\d+|\d+)")
new_df = new_df.drop("Trade", axis=1)
new_df['Pair'] = new_df['Pair'].str.extract(r"([a-zA-Z]+/[a-zA-Z]+)")
new_df[new_df['Pair'].str.contains("XAU/USD") == False]

,Name,Pair,Position,price_level
0,redcat3003,AUD/JPY,SELL,94.07
1,Minky01,GBP/NZD,BUY,1.9210
3,RunForexxRun,GBP/AUD,SELL,1.7341
6,Mrkas,BTC/USD,SELL,23
8,dawnpatrol,EUR/USD,SELL,1.0210
9,Phakawit,EUR/USD,SELL,1.0210
10,parvizizad,EUR/JPY,BUY,137.97
11,Nelo2021,USD/JPY,SELL,135.10
12,RogerJT,EUR/USD,SELL,1.0209
13,RogerJT,EUR/USD,BUY,1.0210


In [64]:
new_df = new_df.dropna(subset=['Pair']) 

In [66]:
new_df.to_csv('raw_data.csv', mode='a', index=False)